## Here we implement the algorithm for the recommendation tool. 

1) We import the data for our wardrobe
2) We import the prediction model
3) We implement the recommendation algorithm

In [2]:
import pandas as pd
import string
import numpy as np
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tqdm import tqdm
from PIL import Image
import cv2
%matplotlib inline

from tensorflow.keras.models import load_model


# STEP 1: We import the data from our wardrobe

In [4]:
# Function that takes a path of an image and returns its array
def preprocess_image_1(im_path):
    img_np = cv2.imread(im_path, cv2.IMREAD_COLOR)
    img_np = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    im = cv2.resize(img_np, (224,224))
    im = im/255  
    return im

In [5]:
# We create a function that takes as input a path of the wardrobe and loads the images to a data dictionary. We use the preprocess function also
categories = ["Tops", "Bottoms", "Shoes", "Bags", "Accessories", "Outerwear", "Onepiece"]
def import_wardrobe(base_dir):
    # A dictionary to hold your image data per category
    data = {category: [] for category in categories}

    for category in categories:
        category_folder = os.path.join(base_dir, category)

        # Ensure the path exists
        if os.path.exists(category_folder):
            for img_file in os.listdir(category_folder):
                # Ensure dealing only with .jpg files or other image types
                if img_file.endswith(".jpg"):
                    img_path = os.path.join(category_folder, img_file)
                    # Use your image preprocessing function here
                    img = preprocess_image_1(img_path)
                    data[category].append(img)

    # Convert lists of images per category into numpy arrays
    for category in categories:
        data[category] = np.array(data[category])
        return data
# We define our path of the wardrobe
   
path = "/home/thanos/Desktop/Sustainable_fashion/data/test_images/test_wardrobe"
# We call the function to import the wardrobe from the path
data = import_wardrobe(path)
    

In [8]:
# Checking our data 
data["Tops"].shape

(4, 224, 224, 3)

# STEP 2: Importing the prediction model we will use to get the scores

In [ ]:
# First we load the prediction model
model = load_model("path_to_saved_model.h5")
# We can display the models summary to check the INPUTS of the model
model.summary()

# STEP 3: Implementing the recommendation algorithm 

In [ ]:
def recommend_outfits(data, categories, recommend_cat):
    # categories = ['Tops', 'Bottoms', 'Shoes','Outwear']
    # recommend_cat = 'Shoes'
    # Create an empty DataFrame to store the outfits and scores
    columns_list = categories + ['Score']
    outfits_df = pd.DataFrame(columns=columns_list)

    # Initial definition of the order of categories for the model
    full_category_order = ["Tops", "Bottoms", "Shoes", "Bags", "Accessories", "Outwear"]


     # Filter the category_order based on the categories provided
    category_order = [cat for cat in full_category_order if cat in categories]

    # Create a dictionary of first items for all categories except the recommendation category
    anchors = {cat: data[cat][0] for cat in categories if cat != recommend_cat}
    x = anchors.keys()
    print(x)

    anchors_exp = {cat: np.expand_dims(anchors[cat], axis=0) for cat in anchors}
    x = anchors_exp.keys()
    print(x) 

    # Predict scores for all items in the recommendation category
    for recommend_item in data[recommend_cat]:
        recommend_item_exp = np.expand_dims(recommend_item, axis=0)

        # Prepare input for the model in the correct order
        model_input = [anchors_exp[cat] if cat in anchors_exp else None for cat in categories]

        # We import the image the we want to recommend in the correct position
        if recommend_cat in category_order:
            model_input[category_order.index(recommend_cat)] = recommend_item_exp

        # # Displaying the image of the recommended item
        # plt.figure()
        # plt.imshow(np.squeeze(model_input[2]))
        # plt.title(f"Recommended {recommend_cat}")
        # plt.axis('off')
        # plt.show()

        # Filter out None values
        model_input = [inp for inp in model_input if inp is not None]
        # Get prediction score
        score = model.predict(model_input)[0]

        # Append the outfit and score to the dataframe
        outfit_data = dict(zip(categories, [anchors.get(cat, recommend_item) for cat in categories]))
        outfit_data['Score'] = score
        outfits_df = outfits_df.append(outfit_data, ignore_index=True)

    outfits_df['Score'] = outfits_df['Score'].astype(float)
    top_outfits_df = outfits_df.nlargest(2, 'Score')

    fig, axs = plt.subplots(2, len(categories), figsize=(5 * len(categories), 10))

    plt.suptitle('Top 2 Recommendations' , fontsize=16, fontweight='bold', y=1.02)

    for idx, (i, outfit) in enumerate(top_outfits_df.iterrows()):
        for col_idx, cat in enumerate(categories):
            axs[idx, col_idx].imshow(outfit[cat])
        axs[idx, col_idx].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
recommend_outfits(data, ['Tops', 'Bottoms', 'Shoes'], 'Shoes')
